In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix



In [ ]:
data = pd.read_csv("../data/Boiler_emulator_dataset_new.csv")

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data["Class"].value_counts()

In [ ]:
data.head()

In [ ]:
data.nunique()

In [ ]:
data["Tsupply"].hist(bins=40)

In [ ]:
trainfull = data[["Fuel_Mdot","Tair","Treturn","Tsupply","Water_Mdot"]]

In [ ]:
target = data["Class"]

In [ ]:
target

In [ ]:
scaler = MinMaxScaler()
scaler = scaler.fit(trainfull)
trainfull[:] = scaler.transform(trainfull)

In [ ]:
target_df= pd.DataFrame(target)

In [ ]:
target_df= pd.DataFrame(target)
target_df.shape

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(target_df)
target_df_encoded = one_hot_encoder.transform(target_df)

In [ ]:
X_train_full , X_test , y_train_full , y_test = train_test_split(trainfull, target_df_encoded, random_state=42)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [ ]:
X_train

In [ ]:
target.head(10)

In [ ]:
target_df_encoded[:10]

In [ ]:
target.nunique()

In [ ]:
n_features = X_train.shape[1]
n_classes = target.nunique()   # quantitative data

In [ ]:
#kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(0.01), use_bias=False)
#model.add(tf.keras.layers.Dropout(0.20))


In [ ]:
tf.random.set_seed(42)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=n_features))
model.add(tf.keras.layers.Dense(1024, activation="swish",))
model.add(tf.keras.layers.Dense(512, activation="swish",))
model.add(tf.keras.layers.Dense(256, activation="swish",))
model.add(tf.keras.layers.Dense(n_classes, activation="softmax"))

In [ ]:
model.summary()


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.002,weight_decay=1e-5)

In [ ]:
model.compile(loss="categorical_crossentropy" #used because the target is number not hot encoded
,optimizer=optimizer,metrics=[tf.keras.metrics.Accuracy(),
                              tf.keras.metrics.Precision(),
                              tf.keras.metrics.Recall(),
                              tf.keras.metrics.F1Score()])

In [ ]:
checkpoint_b = tf.keras.callbacks.ModelCheckpoint("my_checkpoints",
save_weights_only=True, save_best_only = True)

early_stopping_b = tf.keras.callbacks.EarlyStopping(patience=35,
restore_best_weights=True)

In [ ]:
R = model.fit(X_train,y_train,epochs=200, callbacks=[early_stopping_b,checkpoint_b],validation_data=(X_valid,y_valid))

In [ ]:
checkpoint_b.best

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
y_pred_labels

In [ ]:
y_test

In [ ]:
y_test_labels = np.argmax(y_test, axis=1)

In [ ]:
y_test_labels

In [ ]:
confusion_mat = confusion_matrix(y_test_labels,y_pred_labels)
plot_confusion_matrix(confusion_mat)

In [ ]:
R.history.keys()

In [ ]:
# Plot loss
plt.plot(R.history['loss'], label='Training Loss')
plt.plot(R.history['val_loss'], label='Validation Loss')



# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')

# Add a legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
# Plot accuracy
plt.plot(R.history['accuracy'], label='Training accuracy')
plt.plot(R.history['val_accuracy'], label='Validation accuracy')


# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.title('Training and Validation Loss Over Epochs')

# Add a legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
model.save("my_keras_model", save_format="tf")

In [ ]:
tf.random.set_seed(42)
model_2 = tf.keras.models.Sequential()
model_2.add(tf.keras.layers.Input(shape=n_features))
model_2.add(tf.keras.layers.Dense(1024, activation="swish",))
model_2.add(tf.keras.layers.Dense(512, activation="swish",))
model_2.add(tf.keras.layers.Dense(256, activation="swish",))
model_2.add(tf.keras.layers.Dense(n_classes, activation="softmax"))

In [ ]:
model_2.summary()

In [ ]:
optimizer_2 = tf.keras.optimizers.Adam(learning_rate=0.001,weight_decay=1e-5)

In [ ]:
model_2.compile(loss="categorical_crossentropy" #used because the target is number not hot encoded
,optimizer=optimizer_2,metrics=["accuracy", tf.keras.metrics.Precision(),
                              tf.keras.metrics.Recall(),
                              tf.keras.metrics.F1Score()])

In [ ]:
checkpoint_b_2 = tf.keras.callbacks.ModelCheckpoint("my_checkpoints",
save_weights_only=True, save_best_only = True)

early_stopping_b_2 = tf.keras.callbacks.EarlyStopping(patience=35,
restore_best_weights=True)

In [ ]:
R_2 = model_2.fit(X_train,y_train,epochs=500, callbacks=[early_stopping_b_2,checkpoint_b_2],validation_data=(X_valid,y_valid))

In [ ]:
checkpoint_b_2.best

In [ ]:
model_2.evaluate(X_test,y_test)

In [ ]:
y_pred_2 = model_2.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred_labels_2 = np.argmax(y_pred_2, axis=1)

In [ ]:
y_pred_labels_2

In [ ]:
y_test

In [ ]:
y_test_labels = np.argmax(y_test, axis=1)

In [ ]:
y_test_labels

In [ ]:
confusion_mat = confusion_matrix(y_test_labels,y_pred_labels_2)
plot_confusion_matrix(confusion_mat)

In [ ]:
# Plot loss
plt.plot(R_2.history['loss'], label='Training Loss')
plt.plot(R_2.history['val_loss'], label='Validation Loss')



# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')

# Add a legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
# Plot accuracy
plt.plot(R_2.history['accuracy'], label='Training accuracy')
plt.plot(R_2.history['val_accuracy'], label='Validation accuracy')


# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.title('Training and Validation  Over Epochs')

# Add a legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
model_2.save("my_keras_model_2", save_format="tf")